<a href="https://colab.research.google.com/github/iamharkirat/Machine-Learning-Projects/blob/main/satellite_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="j02OP0uADkuhInK3KhGW")
project = rf.workspace("object-detection-in-satellite-images").project("object-detection-in-satellite-images-master-thesis")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=11acbd3875847daa26f2d95cf9315124f27169679f54f609876970fbaa42586b
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Object-Detection-in-Satellite-Images-MAster-Thesis-1 in yolov5pytorch:: 100%|██████████| 21780/21780 [00:04<00:00, 4651.72it/s]


In [ ]:
!pip install pascal-voc-writer
!pip install imageai

import numpy as np 
import pandas as pd 
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from pascal_voc_writer import Writer
import tqdm
import imageai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
tf.__version__

'2.11.0'

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# run it once only, directory structure for YOLO
modelFolder = "/content/images"
os.mkdir(modelFolder +"train") 
os.mkdir(modelFolder +"valid")
os.mkdir(modelFolder +"train/images")
os.mkdir(modelFolder +"train/labels")
os.mkdir(modelFolder +"valid/images")
os.mkdir(modelFolder +"valid/labels")

FileExistsError: ignored

In [ ]:
# training cdv file read
trainCsvPath = '/content/drive/My Drive/sml_project/train_ship_segmentations_v2.csv'
trainImagesPath = modelFolder + 'train/images/'
valImagesPath = modelFolder + 'validation/images/'

imageMasks = pd.read_csv(os.path.join(trainCsvPath))
print(imageMasks.shape[0], 'masks found')
print(imageMasks['ImageId'].value_counts().shape[0])
imageMasks.head()

In [ ]:
modelFolder = "/content/images"
trainImagesPath = modelFolder + 'train/images/'
valImagesPath = modelFolder + 'valid/images/'

In [ ]:
## decode run length encoding decoidng, maks image generation, pascal voc format annotations generation
def decodeRle(rleMask):
    rleMask = rleMask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (rleMask[0:][::2], rleMask[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(768*768, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(768,768).T

def generateMaskImage(masksList):
    maskImage = np.zeros(shape=(768,768))
    for mask in masksList:
        decodedMask = decodeRle(mask)
        maskImage+=decodedMask
    return maskImage
# def pre_process_images():
    
def getBoundingBox(maskImage):
    labels = label(maskImage) 
    coordinates = regionprops(labels)
    return coordinates

def generateTrainData():
    # trainImages = imageMasks.loc[imageMasks.EncodedPixels.isnull()==False]["ImageId"]
    # trainImages = np.unique(trainImages.values)
    for dirname, _, filenames in os.walk( trainImagesPath ):
        trainImages = filenames

    imagePath= modelFolder + "train/images/"
    annotationPath = modelFolder +"train/labels/"
    for i,imageName in enumerate(trainImages):
        print(i)
        image = cv2.imread(trainImagesPath + imageName)
        masksList = imageMasks.loc[imageMasks["ImageId"]==imageName]["EncodedPixels"]
        maskImage = generateMaskImage(masksList)
        coordinates = getBoundingBox(maskImage)
        pascalVoc = Writer(imagePath + imageName, 768, 768)
        for coord in coordinates:
            # cv2.rectangle(image, (coord.bbox[1], coord.bbox[0]), (coord.bbox[3], coord.bbox[2]), (255, 0, 0), 2)
            pascalVoc.addObject('ship', coord.bbox[1], coord.bbox[0], coord.bbox[3], coord.bbox[2])
            
        # cv2.imwrite(os.path.join(imagePath , imageName), image)
        pascalVoc.save(annotationPath + imageName.split('.')[0] + ".xml")
        

def generateValidationData():
    for dirname, _, filenames in os.walk(valImagesPath):
      valImages = filenames
    # valImages = imageMasks.loc[imageMasks.EncodedPixels.isnull()==False]["ImageId"]
    # valImages = np.unique(valImages.values)
    imagePath=modelFolder + "valid/images/"
    annotationPath = modelFolder + "valid/labels/"
    for i, imageName in enumerate(valImages):
        print(i)
        image = cv2.imread(valImagesPath + imageName)
        masksList = imageMasks.loc[imageMasks["ImageId"]==imageName]["EncodedPixels"]
        maskImage = generateMaskImage(masksList)
        coordinates = getBoundingBox(maskImage)
        pascalVoc = Writer(imagePath + imageName, 768, 768)
        for coord in coordinates:
            # cv2.rectangle(image, (coord.bbox[1], coord.bbox[0]), (coord.bbox[3], coord.bbox[2]), (255, 0, 0), 2)
            pascalVoc.addObject('ship', coord.bbox[1], coord.bbox[0], coord.bbox[3], coord.bbox[2])
            
        # cv2.imwrite(os.path.join(imagePath , imageName), image)
        pascalVoc.save(annotationPath + imageName.split('.')[0] + ".xml")

In [ ]:
## generate pascal voc format annotations for trian images
generateTrainData()

In [ ]:
## generate pascal voc format annotations for test images
generateValidationData()

FileExistsError: ignored